#Predição de ataques cardíacos
####Este é um projeto de analise de um conjunto de dados que contem informações pessoais, dados de saúde e habitos de pessoas aleatorias. O objetivo deste estudo é criar um algoritimo de machine learning onde ao informar seus dados, você terá como retorno, se você tem ou não riscos de ter um ataque cardíaco.

##Carregando as bibliotecas

In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier

##Carregando o Data Frame

In [31]:
df_ha = pd.read_csv('heart_attack_prediction_dataset.csv',index_col=0)
df_ha.reset_index(inplace= True)

##Tratamento inicial dos dados e retirada dos dados faltantes caso haja

Retirando do Data Frame **df_ha** as colunas que eu não considero importantes para o estudo

In [32]:
df_ha.drop(['Country','Continent','Hemisphere','Income','Physical Activity Days Per Week','Patient ID','Sedentary Hours Per Day'],axis = 1, inplace= True)

Renomeando as colunas do Data Frame

In [33]:
df_ha.rename({'Age':'Idade','Sex':'Sexo','Cholesterol':'Colesterol','Blood Pressure':'Pressao Sanguinea','Heart Rate':'Frequencia Cardiaca','Family History':'Historico Familiar','Smoking':'Fumante','Obesity':'Obeso','Alcohol Consumption':'Consome Alcool',
              'Exercise Hours Per Week':'Horas de Exercicio Por Semana','Diet':'Estilo de Dieta','Previous Heart Problems':'Problemas Cardiacos Anteriores','Medication Use':'Usa medicamentos','Stress Level':'Nivel de Stress',
              'BMI':'IMC','Triglycerides':'Triglicerides','Sleep Hours Per Day':'Horas de Sono Por Dia','Heart Attack Risk':'Risco de Ataque Cardiaco'}, axis = 1, inplace = True)

Verificando se há dados faltantes no Data Frame

In [34]:
df_ha.isna().sum()

Idade                             0
Sexo                              0
Colesterol                        0
Pressao Sanguinea                 0
Frequencia Cardiaca               0
Diabetes                          0
Historico Familiar                0
Fumante                           0
Obeso                             0
Consome Alcool                    0
Horas de Exercicio Por Semana     0
Estilo de Dieta                   0
Problemas Cardiacos Anteriores    0
Usa medicamentos                  0
Nivel de Stress                   0
IMC                               0
Triglicerides                     0
Horas de Sono Por Dia             0
Risco de Ataque Cardiaco          0
dtype: int64

Como não há nenhuma coluna faltando dados, nosso Data Frame ficou da seguinte forma

In [35]:
df_ha.head()

,Idade,Sexo,Colesterol,Pressao Sanguinea,Frequencia Cardiaca,Diabetes,Historico Familiar,Fumante,Obeso,Consome Alcool,Horas de Exercicio Por Semana,Estilo de Dieta,Problemas Cardiacos Anteriores,Usa medicamentos,Nivel de Stress,IMC,Triglicerides,Horas de Sono Por Dia,Risco de Ataque Cardiaco
0,67,Male,208,158/88,72,0,0,1,0,0,4.168189,Average,0,0,9,31.251233,286,6,0
1,21,Male,389,165/93,98,1,1,1,1,1,1.813242,Unhealthy,1,0,1,27.194973,235,7,0
2,21,Female,324,174/99,72,1,0,0,0,0,2.078353,Healthy,1,1,9,28.176571,587,4,0
3,84,Male,383,163/100,73,1,1,1,0,1,9.828130,Average,1,0,9,36.464704,378,4,0
4,66,Male,318,91/88,93,1,1,1,1,0,5.804299,Unhealthy,1,0,6,21.809144,231,5,0


##Agora irei categorizar alguns dados do Data Frame para deixar mais fácil a identificação dos intervalos

Transformando os dados de pressão sanguínea em float

In [36]:
df_ha['Pressao Sanguinea'] = df_ha['Pressao Sanguinea'].str.replace("/",'.')
df_ha['Pressao Sanguinea'] = pd.to_numeric(df_ha['Pressao Sanguinea'])

Categorização dos dados de Pressão Sanguínea

In [37]:
df_ha.loc[df_ha['Pressao Sanguinea'] <= 105.60, 'Pressão Arterial'] = 'Baixa'
df_ha.loc[(df_ha['Pressao Sanguinea'] > 105.6) & (df_ha['Pressao Sanguinea'] <= 120.8), 'Pressão Arterial'] = 'Ideal'
df_ha.loc[(df_ha['Pressao Sanguinea'] > 120.8) & (df_ha['Pressao Sanguinea'] <= 130.85), 'Pressão Arterial'] = 'Normal'
df_ha.loc[(df_ha['Pressao Sanguinea'] > 130.85) & (df_ha['Pressao Sanguinea'] <= 140.9), 'Pressão Arterial'] = 'Normal Alta'
df_ha.loc[(df_ha['Pressao Sanguinea'] > 140.9) & (df_ha['Pressao Sanguinea'] <= 160.1), 'Pressão Arterial'] = 'Hipertensão Grau 1'
df_ha.loc[(df_ha['Pressao Sanguinea'] > 160.1) & (df_ha['Pressao Sanguinea'] <= 180.11), 'Pressão Arterial'] = 'Hipertensão Grau 2'
df_ha.loc[df_ha['Pressao Sanguinea'] > 180.11, 'Pressão Arterial'] = 'Crise Hipertensiva'

Categorização dos dados de Nivel de Colesterol

In [38]:
df_ha.loc[df_ha['Colesterol'] <= 200, 'Nivel Colesterol'] = 'Normal'
df_ha.loc[(df_ha['Colesterol'] > 200) & (df_ha['Colesterol'] <= 239), 'Nivel Colesterol'] = 'Limítrofe'
df_ha.loc[df_ha['Colesterol'] >= 200, 'Nivel Colesterol'] = 'Alto'

Categorização dos dados de Nivel de Triglicerides

In [39]:
df_ha.loc[df_ha['Triglicerides'] <= 150, 'Nível Triglicerides'] = 'Normal'
df_ha.loc[(df_ha['Triglicerides'] > 150) & (df_ha['Triglicerides'] <= 199), 'Nível Triglicerides'] = 'Limítrofe'
df_ha.loc[(df_ha['Triglicerides'] >= 200) & (df_ha['Triglicerides'] <= 499), 'Nível Triglicerides'] = 'Alto'
df_ha.loc[df_ha['Triglicerides'] >= 500, 'Nível Triglicerides'] = 'Muito Alto'

Categorização do IMC

In [40]:
df_ha.loc[df_ha['IMC'] <= 18.5, 'Classificação IMC'] = 'Abaixo'
df_ha.loc[(df_ha['IMC'] > 18.5) & (df_ha['IMC'] <= 24.9), 'Classificação IMC'] = 'Ideal'
df_ha.loc[(df_ha['IMC'] >= 25) & (df_ha['IMC'] <= 29.9), 'Classificação IMC'] = 'Acima'
df_ha.loc[(df_ha['IMC'] >= 30) & (df_ha['IMC'] <= 34.9), 'Classificação IMC'] = 'Obesidade'
df_ha.loc[(df_ha['IMC'] >= 35) & (df_ha['IMC'] <= 39.9), 'Classificação IMC'] = 'Obesidade Severa'
df_ha.loc[df_ha['IMC'] >= 40, 'Classificação IMC'] = 'Obesidade Morbida'

Categorização da Idade

In [41]:
df_ha.loc[(df_ha['Idade'] >= 18) & (df_ha['Idade'] <= 30), 'Idade_cat'] = '18 a 30'
df_ha.loc[(df_ha['Idade'] > 30) & (df_ha['Idade'] <= 54), 'Idade_cat'] = '31 a 54'
df_ha.loc[(df_ha['Idade'] > 54) & (df_ha['Idade'] <= 70), 'Idade_cat'] = '55 a 70'
df_ha.loc[df_ha['Idade'] > 70, 'Idade_cat'] = '70 +'

Categorização da Frequência Cardiaca

In [42]:
df_ha.loc[df_ha['Frequencia Cardiaca'] <= 69, 'Frequencia Car'] = 'Otima'
df_ha.loc[(df_ha['Frequencia Cardiaca'] > 69) & (df_ha['Frequencia Cardiaca'] <= 76), 'Frequencia Car'] = 'Normal'
df_ha.loc[(df_ha['Frequencia Cardiaca'] > 76) & (df_ha['Frequencia Cardiaca'] <= 83), 'Frequencia Car'] = 'Limitrofe'
df_ha.loc[df_ha['Frequencia Cardiaca'] > 84, 'Frequencia Car'] = 'Ruim'

Categorização das Horas de Exercicio Por Semana

In [43]:
df_ha.loc[df_ha['Horas de Exercicio Por Semana'] <= 2.5, 'Exercicios Semana'] = 'Pouco'
df_ha.loc[(df_ha['Horas de Exercicio Por Semana'] > 2.5) & (df_ha['Horas de Exercicio Por Semana'] <= 6), 'Exercicios Semana'] = 'Ideal'
df_ha.loc[df_ha['Horas de Exercicio Por Semana'] > 6, 'Exercicios Semana'] = 'Otimo'

Transformando a coluna Sexo em valores Booleanos

In [44]:
df_ha['Sexo'] = df_ha['Sexo'].str.replace('Male','1')
df_ha['Sexo'] = df_ha['Sexo'].str.replace('Female','0')
df_ha['Sexo'] = pd.to_numeric(df_ha['Sexo'])

Retirando as colunas que foram categorizadas

In [45]:
df_ha.drop(['Colesterol','Pressao Sanguinea','Triglicerides','IMC','Idade','Frequencia Cardiaca','Horas de Exercicio Por Semana'],axis = 1, inplace= True)

Assim ficou o Data Frame depois de categorizado

In [46]:
df_ha.head()

,Sexo,Diabetes,Historico Familiar,Fumante,Obeso,Consome Alcool,Estilo de Dieta,Problemas Cardiacos Anteriores,Usa medicamentos,Nivel de Stress,Horas de Sono Por Dia,Risco de Ataque Cardiaco,Pressão Arterial,Nivel Colesterol,Nível Triglicerides,Classificação IMC,Idade_cat,Frequencia Car,Exercicios Semana
0,1,0,0,1,0,0,Average,0,0,9,6,0,Hipertensão Grau 1,Alto,Alto,Obesidade,55 a 70,Normal,Ideal
1,1,1,1,1,1,1,Unhealthy,1,0,1,7,0,Hipertensão Grau 2,Alto,Alto,Acima,18 a 30,Ruim,Pouco
2,0,1,0,0,0,0,Healthy,1,1,9,4,0,Hipertensão Grau 2,Alto,Muito Alto,Acima,18 a 30,Normal,Pouco
3,1,1,1,1,0,1,Average,1,0,9,4,0,Hipertensão Grau 2,Alto,Alto,Obesidade Severa,70 +,Normal,Otimo
4,1,1,1,1,1,0,Unhealthy,1,0,6,5,0,Baixa,Alto,Alto,Ideal,55 a 70,Ruim,Ideal
